Extract new sentences
---------------------

In this notebook, we want to extract new sentences from the book [diagne_grammaire](http://wolofresources.org/language/download/diagne_grammaire_wolof.pdf), which contains many examples of Wolof sentences traduce to French. We want to extract them to augment the already sentences we got and for which the treatment is done in the following notebook [extract_sentences](extract_sentences.ipynb). We will create a class which will make working fast. 

### Preprocessing of the book

To extract the sentences from the book, we used the [Wondershare PDFElement](https://pdf.wondershare.net/fr/?gclid=Cj0KCQjwuLShBhC_ARIsAFod4fK-9pyDwLQmwNJYiw0CjsIXCMtvOX9iizTLLYpu52d6Qml12VkAKB0aAqMjEALw_wcB) software again. The author commonly used a colon to separate the Wolof sentences from their French translations (to form translation groups). We identified some problems with the extractions: 
1. The book contains many not interesting sentences separated by a colon
2. Some translation groups are not separated from the rest of the text, and it won't be straightforward to recuperate them directly
3. Some translation groups are too long, and so they are continually at the following line(s)
4. Some translation groups are not complete
5. A colon does not separate some of them

We decide to make the following preprocessing on the book for easier extraction:
- We manually extracted the sentences with the problem numbers 3, 4, and 5 and added on them some corrections: making a translation group to be written in one line; completing some sentences; separating a translation group into multiple ones *; manually writing the translation groups that are not separated by a colon.  
- Identifying and removing the sentences with problem number 1.
- Identifying the translation group with problem number 2 and adding many spaces to correct that problem because most translation groups are separated by more than one space.
- Adding manual corrections to correct the different problems directly from the text file.


*: Some sentences in the translation groups contain one of the following marks: 
- "...": To indicate that the sentence is not complete
- "—" "," ";", "÷": To separate translations of the same sentence
- "(...)" "/" "—": To indicate a possible completion or version of a word or sentence translation. Note that "..." in the parentheses indicate that it contains some text
- ":" "²÷": To separate parts of the same Wolof sentence 

For that group translations, we have different options: 
- Rewrite them manually as different translation groups (more accurate correction)
- Decide to separate them by the identified mark
- Decide to create another translation group with the element to add in a sentence for completion (use more when the marks are parentheses)
- Modify a mark with another mark
- Delete a mark

Some translation groups can contain multiple marks so that multiple corrections will be added. To make the latter, we will need to identify which mark begins first (for example, a "/" can be inside parentheses so that the parentheses will be identified first in the sentence).

**Remark**: The preprocessing of the book can take some days, so we will create a class that will make corrections to a certain number of translation groups and continues at any time if the program is interrupted, for example. 

### Extraction

#### Extract groups

Let us load the manually processed test file and see what it contain.

In [1]:
with open("data/diagne_grammaire_wolof.txt", "r", encoding="utf-8") as f:
    
    txt_file = f.read()
    
print(txt_file)



20         GRAMMAIRE DE WOLOF MODERNE

Cette analyse phonologique procède des principes exposés dans
le Parler de Hauteville ¹. Les phonèmes, comme entités phoniques
indivisibles, y sont définis à partir des contrastes de substances qui
connotent des distinctions de sens.

A. — SYSTÈME CONSONANTIQUE

a) INVENTAIRE DES CONSONNES ORALES ET NASALES.

p   l'identification se fera par rapprochement avec b et m
p/b    paq : coiffure de jeune fille    baq : terre humide
p/m    matt : bois de chauffage      patt : borgne

up : fermer            um : porter malchance

p  est une consonne bilabiale, orale et sourde. Elle se réalise à
partir du contact des lèvres. Elle peut être implosive ou
explosive.

b  identification par opposition à p et m :

p/b   cf. ci-dessus

p/m   bokk : posséder ensemble    mokk : être pulvérisé

lam : bracelet          lap : se noyer
amal : trouver quelque chose  abal : prêter
pour quelqu'un

b  a un même point d'articulation que p. C'est une consonne sonore
et oral

Let us create the extraction class.

In [35]:

# %%writefile wolof-translate/wolof_translate/utils/extract_new_sentences.ipynb
from typing import *
import pandas as pd
import pickle
import re
import os

class NewSentenceExtraction:

    def __init__(self, text: Union[str, None] = None, sent_sep: str = ":", corpus_1: str = "french", corpus_2: str = "wolof", save_directory: str = "data/additional_documents/diagne_sentences/", checkpoint_name: str = "new_sentences"):
        
        self.text = text
        
        self.corpus_1 = corpus_1
        
        self.corpus_2 = corpus_2
        
        self.sep = sent_sep
        
        self.groups = []
        
        self.index = 0
        
        self.save_directory = save_directory
        
        self.checkpoint = checkpoint_name
        
        self.extractions = {corpus_1: [], corpus_2: []}
        
    def __save(self):
        
        checkpoints = {
            # 'extractions': self.extractions,
            'index': self.index,
            # 'groups': self.groups
        }
        
        pd.DataFrame({'groups': self.groups}).to_csv(os.path.join(self.save_directory, 'groups.csv'), index=False)
        
        pd.DataFrame(self.extractions).to_csv(os.path.join(self.save_directory, 'extractions.csv'), index=False)
        
        with open(os.path.join(self.save_directory, self.checkpoint), "wb") as f:
            
            pickler = pickle.Pickler(f)
            
            pickler.dump(checkpoints)
    
    def sep_with_mark(self, group: str, mark: Union[str, None] = None):
        
        raise NotImplementedError
    
    def load(self):
        
        with open(os.path.join(self.save_directory, self.checkpoint), "rb") as f:
            
            depickler = pickle.Unpickler(f)
            
            checkpoints = depickler.load()
        
        try:
            
            self.extractions = pd.read_csv(os.path.join(self.save_directory, 'extractions.csv')).to_dict('list')
        
        except Exception:
            
            pass
             
        self.groups = pd.read_csv(os.path.join(self.save_directory, 'groups.csv'))['groups'].to_list()
        
        self.index = checkpoints['index']
    
    def add_groups(self, new_groups: list):
        
        self.groups += new_groups
        
        self.__save()
        
    def get_groups(self, stop_criterions: list = ["  ", "\n"], comparisons: list = []):
        
        assert not self.text is None
        
        i = 0
        
        a = 0
        
        g = 1
        
        while i < len(self.text):
            
            
            letter = self.text[i]
            
            if letter == self.sep:
                
                print(f"Extraction of group number {g}\n")
                
                b = i - 1 # index of letters before the current letter
                
                a = i + 1 # index of letters after the current letter
                
                corpus_1_s = [] # letters of the left sentence
                
                corpus_2_s = [] # letters of the right sentence
                
                stop = False
                
                for stop_cr in stop_criterions:
                    
                    if self.text[b-len(stop_cr)+1:b+1] == stop_cr:
                        
                        stop = True   
                
                while not stop:
                    
                    corpus_1_s.append(self.text[b])
                    
                    b -= 1
                    
                    stop = False
                
                    for stop_cr in stop_criterions:
                        
                        if self.text[b-len(stop_cr)+1:b+1] == stop_cr:
                            
                            stop = True 
                
                stop = False
                
                for stop_cr in stop_criterions:
                    
                    if self.text[a:a+len(stop_cr)] == stop_cr:
                        
                      stop = True   
                
                while not stop:
                    
                    corpus_2_s.append(self.text[a])
                    
                    a += 1
                    
                    stop = False
                
                    for stop_cr in stop_criterions:
                        
                        if self.text[a:a+len(stop_cr)] == stop_cr:
                            
                            stop = True   
                
                # reverse first sentence
                corpus_1_s.reverse()
                
                # add the sentences
                current_sentence = "".join(corpus_1_s).strip() + f" {self.sep} " + "".join(corpus_2_s).strip()
                
                if "".join(corpus_1_s).strip() != "" and "".join(corpus_2_s) != "":
                    
                    # verify if it is not already manually got
                    not_recuperated = True
                
                    for comparison in comparisons:
                        
                        if current_sentence in comparison:
                            
                            not_recuperated = False
                    
                    # verify if it is not already in the extracted groups
                    for group in self.groups:
                        
                        if current_sentence in group:
                            
                            not_recuperated = False
                    
                    if not_recuperated:
                        
                        self.groups.append(current_sentence.strip())
                        # print(current_sentence)
                
                        g += 1
                    
                        print("Successfully extracted !!\n")
                    
                        print("-----------------\n")
                
                        i = a - 1
                    
                        self.__save()
                
            i += 1
                        
        # print("The groups were successfully recuperated !")
    
    def replace_groups(self, 
                      re_match: str,
                      delete_re: Union[str, None] = None,
                      n_replace_max: int = 1,
                      load: bool = True,
                      save: bool = False, 
                      manual_replace: bool = False,
                      csv_file: str = "founded.csv",
                      force_replace: bool = False):
        
        # we load the data
        if load:
            
            self.load()
        
        # find the groups matching the match regex
        founded = [(i, self.groups[i]) for i in range(len(self.groups)) if re.match(re_match, self.groups[i])]
        
        print(f"Found groups matching the regular expression {re_match} are the followings:\n")
        
        [print(f'- {f[1]}') for f in founded]
        
        print("\n----------------------\n")
        
        # if regex for deletion are provided we replace those that will be found with a max number of replace 
        not_replaced = set()
        
        replaced = set()
        
        result = {}
        
        delete_re_ = input("Do you want to change the deletion' regex expression -> provide one if yes or give empty string ('') if not : ")
        
        if delete_re_ != '':
            
            delete_re = delete_re_
        
        if not delete_re is None or manual_replace:
            
            for i in range(len(founded)):
                
                f = founded[i][1]
                
                index = founded[i][0]
                    
                m_replace = 'n'
                
                if not force_replace and manual_replace:
                    
                    print(f"You will modify the following group:\n {f}")
                    
                    m_replace = input(f"\nDo you want to make a manual replacement of the group {f} -> Yes(y) or No(n). If you want to quit, press q!")
                    
                    if m_replace == 'q':
                        
                        break
                    
                    while not m_replace in ['y', 'n']:
                                    
                        replace_r = input(f"You must provide a response between Yes(y), No(n)!")
                
                    if m_replace != 'n':

                        print(f"The manual modification of the group\n {f}\n is done in the following file: {csv_file}\n!If you want to provide multiple new groups please make them in different lines")
                                    
                        finish = 'n'
                        
                        pd.DataFrame({'to_modify':[f]}).to_csv(csv_file, index = False)

                        while finish == 'n':
                            
                            finish = input("Did you finish to replace -> No(n) if you didn't finish yet, click any another key if Yes(y) : ")
                        
                        f = pd.read_csv(csv_file)['to_modify'].to_list()
                        
                    print("\n--------\n")
                                            
                if not delete_re is None and m_replace in ['n', '']:
                    
                    to_replace = set(re.findall(delete_re, f))
                    
                    replace_r = None
                    
                    for r in to_replace:
                        
                        if force_replace:
                            
                            f = f.replace(r, '', n_replace_max)
                            
                            replaced.add(f)
                        
                        else:
                            
                            replace_r = input(f"Do you want to replace the {r} string in the group:\n {f} ? Yes(y) or No(n). If you want to quit, press q!")
                            
                            if m_replace == 'q':
                        
                                break
                            
                            while not replace_r in ['y', 'n']:
                                
                                replace_r = input(f"You must provide a response between Yes(y) and No(n)!")
                            
                            if replace_r == 'y':
                            
                                f = f.replace(r, '', n_replace_max)
                                
                                replaced.add(f)
                            
                            else:       
                                
                                not_replaced.add(f)
                    
                    if not replace_r is None and replace_r == 'q':
                        
                        break
                                                 
                if isinstance(f, str): 
                    
                    f = [f.strip()]
                
                else:
                    
                    f = [f_.strip() for f_ in f]
                    
                try:
                
                    self.groups = self.groups[:index] + f + self.groups[index+1:]
                
                except IndexError:
                    
                    self.groups = self.groups[:index] + f 

                if len(f) > 1 and i != len(founded) - 1:
                    
                    for j in range(i + 1, len(founded)):
                        
                        founded[j] = (founded[j][0] + len(f) - 1, founded[j][1]) 
                    
                result[index] = f
                
            if save:
            
                print("Final result:")
                
                [print(v) for r, v in result.items()]
                
                save_result = input("Do you want to save the result ? Yes(y) or No(n)")
                
                while not save_result in ['y', 'n']:
                                    
                    replace_r = input(f"You must provide a response between Yes(y) or No(n) !")
                
                if save_result == 'y':
                    
                    self.__save()
        
        return {'founded': founded, 'result': result, 'replaced': replaced, 'not_replaced': not_replaced}
    
    def extraction_commands(self, 
                        add_end_mark_cmd: str = "a", 
                        pass_cmd: str = "p",
                        add_end_mark_on_all: str = "l",
                        add_upper_cmd: str = "u",
                        add_upper_on_all: str = "o",
                        sep_cmd: str = "_",
                        quit_cmd: str = "q",
                        ):
                    
        # recuperate the current command
        cm = input(f"Choose one of the following commands: \n- {add_end_mark_cmd}+group_nb1,group_nb2:mark|group_nb3,group_nb4:mark|...(or group_nb1-group_nbn:mark) : To add end mark on specific groups\
                \n- {add_end_mark_on_all}+mark : To add end mark of all groups, \n- {add_upper_cmd}+group_nb1,group_nb2,group_nb3,group_nb4,...(or group_nb1-group_nbn) : To uppercase the first letter of specific groups\
                    \n- {add_upper_on_all} : To uppercase the first letter of all the groups\
                        \n- {pass_cmd} : To accept all of the groups\
                            \n- {quit_cmd} : To stop the process\
                                \n- You can combine all two commands by underscore {sep_cmd} excepted for the two last commands !")
        
        cms = cm.split(sep_cmd)
        
        error = False
        
        if len(cms) == 2: 
            
            p_cm = [cms[0].split("+")[0], cms[1].split("+")[0]]

            if pass_cmd in p_cm or quit_cmd in p_cm or sep_cmd in p_cm:
                
                print(f"You cannot provide {pass_cmd}, {quit_cmd} or {sep_cmd} in combined commands !")
                
                error = True
            
            elif (p_cm[0] in [add_end_mark_cmd, add_end_mark_on_all] and p_cm[1] in [add_upper_cmd, add_upper_on_all]) or (
                p_cm[0] in [add_upper_cmd, add_upper_on_all] and p_cm[1] in [add_upper_cmd, add_upper_on_all]
            ):
                
                print("You cannot combine the same type of command: Type of commands are 'end mark' and 'upper'")
            
        elif len(cms) == 1:
            
            if not cms[0].split('+')[0] in [add_end_mark_cmd, add_end_mark_on_all, add_upper_cmd, add_upper_on_all, pass_cmd, quit_cmd]:
                
                print("You didn't provide a right command ! Please retry")
                
                error = True 
        
        else:
            
            print("You cannot provide more than 2 or 0 commands !")
            
        return cms, error
            
         
    def split_group(self, group: Union[list, str]):
        # we base on the colon critter to split the groups
        
        if isinstance(group, str):
            
            group = [group]
        
        sents = {
            self.corpus_1: [],
            self.corpus_2: []
            }
        
        for g in group:
            
            splits = g.split(':')
                
            middle = len(splits)//2
            
            cp1_corpus = "".join(splits[:middle])
            
            cp2_corpus = "".join(splits[middle:])

            sents[self.corpus_1].append(cp1_corpus.strip())
            
            sents[self.corpus_2].append(cp2_corpus.strip())
        
        return sents

    def add_end_mark(self, batch: dict, command: str):
        
        cm = command
        
        # recuperate the marks with groups and apply the transformations
        tfs = cm.split('|')
        
        for tf in tfs:
            
            if '-' in tf:
                
                groups = tf.split(':')[0].split('-')
                
                groups = list(range(int(groups[0]), int(groups[1]) + 1))
            
            else:
                
                groups = [int(nb) for nb in tf.split(':')[0].split(',')]
            
            mark = tf.split(':')[1]
            
            for nb in groups:
                
                batch[self.corpus_1][nb-1] += mark
                
                batch[self.corpus_2][nb-1] += mark
        
        return batch
    
    def add_upper(self, batch: dict, command: str):
        
        cm = command
        
        # recuperate the marks with groups and apply the transformations
        tfs = cm.split('|')
        
        for tf in tfs:
        
            # recuperate the marks with groups and apply the transformations
            if '-' in tf:
                    
                groups = tf.split('-')
                
                groups = list(range(int(groups[0]), int(groups[1]) + 1))
            
            else:
                
                groups = [int(nb) for nb in tf.split(',')]
        
            for nb in groups:
                
                batch[self.corpus_1][nb-1] = batch[self.corpus_1][nb-1][0].upper() + batch[self.corpus_1][nb-1][1:] 
                
                batch[self.corpus_2][nb-1] = batch[self.corpus_2][nb-1][0].upper() + batch[self.corpus_2][nb-1][1:] 
        
        return batch
    
    def inner_command(self, batch: dict):
        
        cp1_sents = batch[self.corpus_1]
        
        cp2_sents = batch[self.corpus_2]
                
        for i in range(0, len(batch[self.corpus_1])):
            
            cp1_sent = cp1_sents[i]
            
            cp2_sent = cp2_sents[i]
            
            if re.match('.*Mark\[.*\].*', cp2_sent):
                
                mark = re.findall('Mark\[.*\]', cp2_sent)[0]
                
                mark = mark.replace('Mark[', '').replace(']', '')
                
                cp1_sent = cp1_sent + mark
                
                cp2_sent = re.sub('Mark\[.*\]', '', cp2_sent, 1) + mark
                
            if re.match('.*Upper', cp2_sent):
                
                cp1_sent = cp1_sent[0].upper() + cp1_sent[1:]
                
                cp2_sent = cp2_sent[0].upper() + re.sub('Upper', '', cp2_sent, 1)[1:]

            cp1_sents[i] = cp1_sent
            
            cp2_sents[i] = cp2_sent
            
        batch[self.corpus_1] = cp1_sents
        
        batch[self.corpus_2] = cp2_sents

        return batch

    def extract_sentences(self, 
                          group_range: Union[tuple, None] = None, 
                          add_end_mark_cmd: str = "a", 
                          pass_cmd: str = "p",
                          add_end_mark_on_all: str = "l",
                          add_upper_cmd: str = "u",
                          add_upper_on_all: str = "o",
                          sep_cmd: str = "_",
                          quit_cmd: str = "q",
                          batch_size: int = 30,
                          load: bool = True,
                          save: bool = False,
                          csv_file: str = "batch.csv",
                          last_checkpoint: bool = True
                          ):
        
        
        # we load the data
        if load:
            
            self.load()
            
        # the group range is equal to a tuple containing the last saved index and the index of the last element in the list of groups
        # indices if nothing is given
        if last_checkpoint:
            
            if group_range is None: group_range = (self.index, len(self.groups)-1)
        
        else: 
            
            raise ValueError("You must provide a group range if last checkpoint is to False !")
        
        # change the number of displayed lines
        pd.options.display.max_rows = batch_size
        
        groups =  self.groups[group_range[0]:group_range[1]+1]
        
        # initialize the sub corpora
        sub_corpora = {
            self.corpus_1: [],
            self.corpus_2: []
            }
        
        i = 0
        
        # for each batch we will add the groups in a csv file and take a command
        for b in range(0, len(groups), batch_size):
            
            
            # recuperate a batch
            batch_ = groups[b: b + batch_size]
            
            # recuperate the index
            self.index += len(batch_)
            
            # split each group into two sentences and transform the obtained dictionary to a DataFrame
            batch = self.split_group(batch_)

            pd.DataFrame(batch).to_csv(csv_file, index=False)
            
            print(f"Which of the groups of batch number {i+1} do you consider to be complete sentences (see the file {csv_file}) ?")
            
            error = False
            
            cms = []
            
            try:
                
                cms, error = self.extraction_commands(add_end_mark_cmd, pass_cmd, add_end_mark_on_all, add_upper_cmd, add_upper_on_all, sep_cmd, quit_cmd)
            
            except Exception:
                
                print("You didn't provide a right group number !")
                
                error = True
            
            while error:
                
                error = False
                
                try:
                
                    cms, error = self.extraction_commands(add_end_mark_cmd, pass_cmd, add_end_mark_on_all, add_upper_cmd, add_upper_on_all, sep_cmd, quit_cmd)
                
                except IndexError:
                    
                    print("You didn't provide a right group number !")
                    
                    error = True
            
            # recuperate the batch
            batch = pd.read_csv(csv_file).to_dict('list')
            
            # add corrections
            batch = self.inner_command(batch)
            
            cm_type = ""
            
            quit_ = "n"
            
            for cm in cms:
                
                cm_type = cm.split('+')[0]
            
                if cm_type == add_end_mark_cmd:
                    
                    batch = self.add_end_mark(batch, cm.split("+")[1])
                
                elif cm_type == add_end_mark_on_all:
                    
                    mark = cm.split("+")[1]
                    
                    batch = self.add_end_mark(batch, ",".join([str(nb) for nb in range(1, len(batch[self.corpus_1]) + 1)]) + f":{mark}")
                
                elif cm_type == add_upper_cmd:
                    
                    batch = self.add_upper(batch, cm.split("+")[1])
                
                elif cm_type == add_upper_on_all:
                    
                    batch = self.add_upper(batch, ",".join([str(nb) for nb in range(1, len(batch[self.corpus_1]) + 1)]))
                
                elif cm_type == quit_cmd:
                    
                    quit_ = input("Are you sure you want to quit: Yes(y) or No(n)")
                    
                    while not quit_ in ['y', 'n']:
                        
                        quit_ = input("Are you sure you want to quit: Yes(y) or No(n)")

                    if quit_ == 'y':
                        
                        break
                
                print("\nBatch result")
                
                print(pd.DataFrame(batch).head(batch_size))
            
            print("\n--------------------\n\n")
            
            # add the batch to the sub corpora
            sub_corpora[self.corpus_1].extend(batch[self.corpus_1])
            
            sub_corpora[self.corpus_2].extend(batch[self.corpus_2])

            if cm_type == quit_cmd and quit_ == 'y':
                
                break
            
            else:
                
                if save:
                
                    save_ = input("Do you want to save the result ? Yes(y) or No(n)")
                    
                    while not save_ in ['y', 'n']:
                        
                        save_ = input("Do you want to save the result ? Yes(y) or No(n)")
                    
                    if save_ == 'y':
                        
                        self.extractions[self.corpus_1].extend(batch[self.corpus_1])
                        
                        self.extractions[self.corpus_2].extend(batch[self.corpus_2])
                        
                        self.__save()
            
            i += 1
        
        print("Finished !")
        
    def remove_duplicated_sentences(self, save: bool = False):
        
        # we load the data
        self.load()
        
        # use pandas to delete the duplicated rows
        extractions = pd.DataFrame(self.extractions)
        
        extractions.drop_duplicates(inplace=True)
        
        self.extractions = extractions.to_dict('list')
        
        # save the sentences
        if save:
            
            self.__save()
            

Let us recuperate the manually the extracted groups.

In [36]:
with open("data/diagne_manual_recuperation.txt", "r", encoding="utf-8") as f:
    
    comparisons = f.read().strip()

In [37]:
# print manually extracted groups' text
comparisons

"anal : ramasser des ordures pour quelqu'un   \namal ! : obtenir quelque chose pour quelqu'un  \nwànal : montrer quelque chose (à quelqu'un de la part de quelqu'un d'autre)\ndac : toucher, entrer en contact avec\ndad : user par frottement\nxaañoo : se briser la tête mutuellement\nraf : clignoter, bouger de façon subreptice\nlaaw : prendre dans un filet\ndar : écorché, usé par frottement\nlaw : s'étendre \nlaf : bande d'étoffe grimpante\nsiifal : accaparer quelque chose pour quelqu'un\nfay : abandonner le faj domicile conjugal\nxamp : mordre à pleines dents\nsat : battre quelqu'un à plusieurs\nsañ : faire preuve d'audace excessive, manque de gêne\nlaõ : s'exiler plus ou moins définitivement\nxajal: faire de la place à quelqu'un \nxajjal : frayer un chemin à quelqu'un\ndagu: adopter une attitude de serviteur vis-à-vis de quelqu'un\njëxi : être sur le point de s'épuiser\njoor : terrain sablonneux, mais aussi nom de personne\nmaõkoo : être de connivence  \nweõgalu : pencher d'un côté\nteõx

Let us separated the groups by the brake line "\n".

In [38]:
comparisons = comparisons.split("\n")

In [39]:
# print again
comparisons

["anal : ramasser des ordures pour quelqu'un   ",
 "amal ! : obtenir quelque chose pour quelqu'un  ",
 "wànal : montrer quelque chose (à quelqu'un de la part de quelqu'un d'autre)",
 'dac : toucher, entrer en contact avec',
 'dad : user par frottement',
 'xaañoo : se briser la tête mutuellement',
 'raf : clignoter, bouger de façon subreptice',
 'laaw : prendre dans un filet',
 'dar : écorché, usé par frottement',
 "law : s'étendre ",
 "laf : bande d'étoffe grimpante",
 "siifal : accaparer quelque chose pour quelqu'un",
 'fay : abandonner le faj domicile conjugal',
 'xamp : mordre à pleines dents',
 "sat : battre quelqu'un à plusieurs",
 "sañ : faire preuve d'audace excessive, manque de gêne",
 "laõ : s'exiler plus ou moins définitivement",
 "xajal: faire de la place à quelqu'un ",
 "xajjal : frayer un chemin à quelqu'un",
 "dagu: adopter une attitude de serviteur vis-à-vis de quelqu'un",
 "jëxi : être sur le point de s'épuiser",
 'joor : terrain sablonneux, mais aussi nom de personne',

Let us initialize the extraction class and extract the group in the book different from those already extracted.

In [40]:
new_sent_extraction = NewSentenceExtraction(txt_file)


In [8]:
new_sent_extraction.get_groups(comparisons=comparisons)

Extraction of group number 1

Successfully extracted !!

-----------------

Extraction of group number 2

Successfully extracted !!

-----------------

Extraction of group number 3

Successfully extracted !!

-----------------

Extraction of group number 4

Successfully extracted !!

-----------------

Extraction of group number 5

Successfully extracted !!

-----------------

Extraction of group number 6

Successfully extracted !!

-----------------

Extraction of group number 7

Extraction of group number 7

Successfully extracted !!

-----------------

Extraction of group number 8

Successfully extracted !!

-----------------

Extraction of group number 9

Successfully extracted !!

-----------------

Extraction of group number 10

Successfully extracted !!

-----------------

Extraction of group number 11

Successfully extracted !!

-----------------

Extraction of group number 12

Successfully extracted !!

-----------------

Extraction of group number 13

Extraction of group numb

Let us print some of the groups.

In [9]:
# new_sent_extraction.groups[-1000:]

Let us add the manually extract groups in the already extracted groups.

In [10]:
new_sent_extraction.add_groups(comparisons)

#### Modify the groups

It is time to add modification on the groups. We will make modification using the regular expressions. We will investigate the groups containing the following expression or marks:

1. groups beginning with a number surrounded by parentheses: re = \(\d+\).\*
2. groups with anti-slashes: re = .\*\\.\*
3. groups with comma: re = .\*,.\*
4. groups containing number: re = .\*\d+.\*
5. groups containing the "²÷" sign: re = .\*²÷.\*
6. groups containing the "²" or "¹" footnote numbers (we have already removed the number three): re = .\*[²¹]+.\*
7. groups containing the semicolons: re = .\*;.\*
8. groups containing the "÷" sign: re = .\*÷.\*
9. groups containing ems: re = .\*—.\*
10. groups containing more than one colon: re = .\*:.\*:.\*
11. groups containing parentheses: re = .\*(.\*).\*

**Regex 1** -->

In [11]:
result = new_sent_extraction.replace_groups("(\(\d+\)).*", force_replace=True, save=True)

Found groups matching the regular expression (\(\d+\)).* are the followings:

- (1) nit ku góor la : c'est un homme
- (2) nit ku baax la : c'est un homme généreux
- (1) dem õga ? : tu as été ?
- (2) déedéed ! : non !
- (2) xanaa ! : bien sur !
- (2) xëyna ! : peut-être ! etc...
- (1) Loo gis ci biti : qu'as-tu vu dehors ?
- (2) nit : (un) homme
- (1) kooy waxal ? : à qui parles-tu ?
- (2) buur : (au) roi
- (2) Ndar : à Saint-Louis
- (1) ki kan la ? : qui est-ce celui-ci ?
- (2) jaambur : un quidam
- (1) loo bëgg def ? : que veux-tu faire ?
- (2) dem : partir
- (1) looy ñaan ? : tu pries pour quoi ?
- (2) rafet : (pour la) beauté
- (1) ku ñëw ? : qui est venu ?
- (2) kenn : personne
- (1) ku wax ? : qui a parlé ?
- (2) kookuu : celui-là
- (1) neex na ? : c'est bon ?
- (2) cam ! : pouah !
- (1) baax na ? : c'est bon ?
- (2) ex ! etc... : oh !
- (1) ana góor gi ? : où est le Monsieur ?
- (2) dem : na : il est parti
- (1) xoolal gii garab : regarde cet arbre
- (2) guy : la : c'est un boaba

**Regex 2** -->

In [17]:
result = new_sent_extraction.replace_groups(".*/.*", save=True, manual_replace=True, csv_file="founded.csv", load=False)

Found groups matching the regular expression .*/.* are the followings:

- na woon : de la manière dont ce fut, / manière / fuut
- leneen loolee woon : cet autre fait-là, chose autre / celle-là / fut
- moo di loola : quel démon ! / lui / être / cela
- dafa di leneen : c'est qu'il s'agit d'autre chose, c'est / autre chose
- góor : gi dem : l'homme qui est parti (l'homme / lequel / s'en alla)
- kookule la : soo demee : c'est celui-là / peut-être
- keneen ki la : defe naa : c'est l'autre / je crois
- foofu góor gi dem fu rafet la : l'endroit où l'homme est parti est beau, là / l'homme / parti / là / beau / est
- fépp fu mu jëm foofu jàmm dana fa wacc : partout où il ira la paix descendra là, partout où / il descend / là / paix / descendra

----------------------

You will modify the following group:
 na woon : de la manière dont ce fut, / manière / fuut
The manual modification of the group
 na woon : de la manière dont ce fut, / manière / fuut
 is done in the following file: founded.csv
!I

**Regex 3** -->

In [14]:
result = new_sent_extraction.replace_groups(".*,.*", save=True, manual_replace=True, csv_file="founded.csv")

Found groups matching the regular expression .*,.* are the followings:

- daj : atteindre, toucher
- daõal : tendre, rendre tendu
- siit : couler, suinter
- fatu : s'abriter, s'enfermer chez soi
- fad : abriter, ranger
- dad : tailler, polir
- jëf : fait, acte
- kuu : celui-là, que voilà
- giir : lignée, descendance
- xer : berge, rivage
- ëpp : dépasser les limites, excessif
- uu kuu : celui-ci, tout près
- ki : cet, que voici
- taf : coller, afficher
- at-té : séparer, juger
- ree-waa-le : rire en même temps de, feindre en riant
- jigéen : femme, etc.
- wàcc : abandonner, etc.
- par : allié, alliance
- jolof : pays du Lof, pays wolof
- kaddu : parole, tonnerre
- ndem : le fait de partir, départ
- safaay : l'acidité, le piquant
- demin : manière d'aller, l'état
- waxin : façon de parler, accent
- waxkat : bavard, locuteur
- sëgg : se courber, attendre
- xamadi : ignorer, ignorance, insolence
- goreedi : manquer de noblesse, indignité
- mosal : faire goûter, pourboire
- gistal : faire 

**Regex 4** -->

In [8]:
result = new_sent_extraction.replace_groups(".*\d+.*", save=True, csv_file="founded.csv")

Found groups matching the regular expression .*\d+.* are the followings:

- géew bi day taxaw walla 1 ma dem : le cercle se forme ou je m'en vais

----------------------

Final result:
["géew bi day taxaw walla  ma dem : le cercle se forme ou je m'en vais"]


**Regex 5** -->

In [9]:
result = new_sent_extraction.replace_groups(".*²÷.*", save=True, csv_file="founded.csv", force_replace=True)

Found groups matching the regular expression .*²÷.* are the followings:

- ku ²÷ dem : qui est parti ?
- góor gu ²÷ dem : quel homme est parti ?
- moo ²÷ dem : c'est lui qui est parti ?
- góor gi ²÷ nit la : l'homme est sain d'esprit
- góor gi ²÷ nitu Ndar la : l'homme est un Saint-Louisien
- góor gi ²÷ di xel doõõ : cet homme n'est qu'intelligence
- berëb yi ²÷ berëb yu yaa lañu : ces lieux sont de vastes étendues
- góor gi ²÷ kenn la : l'homme est unique
- kooku ²÷ keneen la : celui-là c'est un autre
- berëb bi ²÷ du foofu : l'endroit ce n'est pas par-là
- ndaje mi ²÷ ci biir la : la réunion c'est à l'intérieur
- ndaje mi ²÷ du gannaaw ëlëk ? : la réunion ce n'est pas après-
- góor gi ²÷ dem : le Monsieur est parti
- mu ²÷ dem : (mais) il est parti
- ma ngii ²÷ dem : voilà qu'il est parti
- góor ñi ²÷ deekoon nanu : les gens en eussent été morts
- maa ²÷ demulkoon : c'est moi qui n'aurais pas été
- góor ²÷ demul : (un) homme n'a été (nul n'a été)
- góor gi ²÷ demul : l'homme n'a pas 

**Regex 6** -->

In [10]:
result = new_sent_extraction.replace_groups(".*¹+.*", save=True, csv_file="founded.csv", force_replace=True)

Found groups matching the regular expression .*¹+.* are the followings:

- Séen naa ak nit : J'ai aperçu un homme (aw nit) ¹
- lëf lenn réerul ¹ : rien ne s'est égaré
- seneen ¹ ndaw laa bëgg : c'est une autre jeune fille que je veux
- góor gi dana demi ¹ : l'homme ira
- dafa di ¹ nitu tay : c'est un homme de son époque
- góor gi nee mi õgi fi : l'homme affirme qu'il est là ¹
- õga dem la ñooñu bëgg ¹ : que tu partes c'est ce que ceux-là veulent
- Samba : mi ñëw ? ¹ : Samba : qui est venu ?
- li : waa ji wax ¹ : ce que : cet homme a dit
- man ¹ : õga woo : c'est moi (que) tu appelles
- koo ¹ jox xaalis ? : à qui as-tu donné de l'argent ?

----------------------

Final result:
["Séen naa ak nit : J'ai aperçu un homme (aw nit)"]
["lëf lenn réerul : rien ne s'est égaré"]
["seneen ndaw laa bëgg : c'est une autre jeune fille que je veux"]
["góor gi dana demi : l'homme ira"]
["dafa di nitu tay : c'est un homme de son époque"]
["góor gi nee mi õgi fi : l'homme affirme qu'il est là"]
["õga dem

In [11]:
result = new_sent_extraction.replace_groups(".*².*", save=True, csv_file="founded.csv", force_replace=True)

Found groups matching the regular expression .*².* are the followings:

- góor gi : mu àndi ² : l'homme qu'il a amené
- Samba ² : õga woo : c'est Samba que tu appelles
- ci biir ² : õga wax : c'est à l'intérieur que tu dis
- yaw la : bëgg ² : c'est toi (qu')elle veut

----------------------

Final result:
["góor gi : mu àndi : l'homme qu'il a amené"]
["Samba : õga woo : c'est Samba que tu appelles"]
["ci biir : õga wax : c'est à l'intérieur que tu dis"]
["yaw la : bëgg : c'est toi (qu')elle veut"]


**Regex 7** -->

In [12]:
result = new_sent_extraction.replace_groups(".*;.*", save=True, csv_file="founded.csv", manual_replace=True)

Found groups matching the regular expression .*;.* are the followings:

- coy : variété d'oiseau ; coyy : rouge écarlate ; noyyi : respirer
- sawu : urinoir ;
- Jaw ; Diaw : nom de personne ; jaww : atmosphère
- jiitu : précéder ; fit-al : mettre en tête
- samp : planter ; cemp : variété d'herbe aromatique ;
- def : faire ; jëf : acte ; jëfkat : homme d'action
- boot : porter sur le dos ; bootaay : manière de porter sur le dos ;
- dog : couper ; dogal : pourboire ; ndogal : destin
- sopp : estimer ; soppal : estimer à cause de quelqu'un ;
- wax : dire ; weddi : nier ; weddikat : qui nie
- neex : agréable ; naqadi : désagréable ; naqadil : rendre
- dog : couper ; dogalante : se faire de petits cadeaux ; ndogalante : concession mutuelle
- jiitu : précéder ; jiital : mettre en tête ; jiitëlante : se mettre en rang ;
- wax : parler ; waxaale : marchander ; waxaalelu : faire marchander ;
- jéem : essayer ; jëmentu : s'exercer ; jëmentalukaay : terrain d'essai
- jiit qui donne : jiitu : être

**Regex 8** -->

In [18]:
result = new_sent_extraction.replace_groups(".*÷.*", save=True, csv_file="founded.csv", manual_replace=True)

Found groups matching the regular expression .*÷.* are the followings:

- dee-o ÷ deewoo : mourir ensemble
- ji-i ÷ jiyi : aller semer
- jambuur : homme libre ÷ jambuur-jambuur : habitant de la région des
- kërbët : pas de cheval ÷ kërbët kërbët : galop de cheval
- na ma ÷ naa : que je....

----------------------

You will modify the following group:
 dee-o ÷ deewoo : mourir ensemble
The manual modification of the group
 dee-o ÷ deewoo : mourir ensemble
 is done in the following file: founded.csv
!If you want to provide multiple new groups please make them in different lines

--------

You will modify the following group:
 ji-i ÷ jiyi : aller semer
The manual modification of the group
 ji-i ÷ jiyi : aller semer
 is done in the following file: founded.csv
!If you want to provide multiple new groups please make them in different lines

--------

You will modify the following group:
 jambuur : homme libre ÷ jambuur-jambuur : habitant de la région des
The manual modification of the group
 

**Regex 9** -->

In [8]:
result = new_sent_extraction.replace_groups(".*—.*", save=True, csv_file="founded.csv", manual_replace=True)

Found groups matching the regular expression .*—.* are the followings:

- fab — fap : prendre
- fat — fad : ranger
- fac — faj : soigner
- tak — tag : se poser
- fap — fab : prendre
- dek — deg : épine degu àll : épine de brousse
- cëriñ : islam — l'état de musulman
- nit dem na : (un) homme est parti — quelqu'un est parti
- su dee dem : alors qu'il sera en train de partir — s'il part
- su demee : s'il est parti — une fois là-bas — s'il part
- su dee dem : quand il partira — une fois en chemin
- kenn — u nit : un seul homme
- ñenn — u nit : certaines gens (quelques-uns parmi les hommes) yenn
- araf céex — ñaari araf yu niróo te ànd : géminée.
- waa jii ag waa joojale duñu benn : Cet homme-ci — devant nous — et cet homme-là (éloigné dans le temps) ne sont pas les mêmes
- ñii dañu demul xanaa : ceux-ci ne partent peut-être pas — ne sont peut-être pas partis

----------------------

You will modify the following group:
 fab — fap : prendre
The manual modification of the group
 fab — fap :

**Regex 10** -->

In [8]:
result = new_sent_extraction.replace_groups(".*:.*:.*", save=True, csv_file="founded.csv", manual_replace=True)

Found groups matching the regular expression .*:.*:.* are the followings:

- samay xarit : des amis à moi : mes amis
- nit ku góor : être humain (lequel) (mâle) : un homme
- nit ñu góor : être humain (lesquels) (mâles) : hommes
- kaay : viens kaay : est une forme qui n'apparaît dans aucun autre
- dem : na : il est parti
- guy : la : c'est un boabab
- kile : la : c'est celui-là
- ci : biir : à l'intérieur
- õgoor : dem : (mais) NGor est parti
- nit : ki ñëw : l'homme qui est venu
- dem na : ci biir : il est allé à l'intérieur
- bëgg na : õga dem : il veut que tu viennes
- dama bëgg : dem : c'est que je veux partir
- dafa bëgg : xaalis : c'est qu'il veut de l'argent
- dafa bañ : rewande : c'est qu'il déteste l'impolitesse
- bëgg na : gëléem : il aime le chameau
- gis na : ndaw : il a vu un messager
- góor gi gisul : xale boobale woon : l'homme n'a pas vu cet enfant-là
- négël : xale bu rafet bi : attends la belle jeune fille
- sërin bi bëgg na : mayewoon alalam ji ba mënkoon yalwaan tay 

**Regex 11** -->

In [8]:
result = new_sent_extraction.replace_groups(".*\(.*\).*", save=True, csv_file="founded.csv", manual_replace=True)

Found groups matching the regular expression .*\(.*\).* are the followings:

- wàtal : traîner (pour quelqu'un)
- sap : chanter (coq)
- an : enlever (ordures)
- sañc : bâtir (une ville)
- ree-aat-le : rire encore avec quelqu'un (pour lui)
- saf : piquant (goût)
- Ndar : Saint-Louis (ville)
- nit bonul : (l')homme n'est pas mauvais
- gis naa gaynde : j'ai vu (un) lion
- nit la : c'est (un) homme
- xoolal mbër mu : regarde ce (semblant de) lutteur
- gis õga nit ka : tu as vu l'homme (au passé)
- Séen naa ak nit : J'ai aperçu un homme (aw nit)
- Xar moomu man õga wax ? : tu parles de (ce) quel mouton ?
- Xar mii man ? : (le) quel mouton ?
- Xar yii yan õga jënd ? : Quels moutons (ceux-là) avais-tu achetés ?
- Xar yile yan ? : Quels (ceux-là) moutons ?
- Ma japp nag yee yan ? : Que j'attrape quelles vaches (celles-là) ?
- Mu japp nag yooyu yan ? : Qu'il attrappe quelles vaches (celles-là) ?
- Boobu néeg ban õga wax ? : Tu parles de quelle maison (là) ?
- Bii néeg ban õga wax ? : Tu parles 

#### Extract sentences

It is time to extract the sentences. We will recuperate the sentences considering batches to add additional modifications if necessary, like the following:
- Add end marks.
- Modify the first letter of groups to upper case.
- Modify the sentences directly from a CSV file.

We can save a result from a batch, stop the process, and continue another when no index is indicated. The index will automatically be saved.

In [85]:
new_sent_extraction.extract_sentences(group_range=(0, 1000), save=True, batch_size=100)

Which of the groups of batch number 1 do you consider to be complete sentences (see the file batch.csv) ?

Batch result
    french                        wolof
0      paq      coiffure de jeune fille
1      baq                 terre humide
2     matt            bois de chauffage
3     patt                       borgne
4       up                       fermer
5       um             porter malchance
6     bokk            posséder ensemble
7     mokk               être pulvérisé
8      lam                     bracelet
9      lap                     se noyer
10    amal        trouver quelque chose
11    abal                       prêter
12    natt                      mesurer
13     gëm                       croire
14     gën                être meilleur
15     fac                      soigner
16     pac                         soin
17    suuf                        sable
18    suup                      teindre
19     daf                       souder
20     dap                    rejoindre


In [88]:
new_sent_extraction.extract_sentences(group_range=(300, 1000), save=True, batch_size=100)

Which of the groups of batch number 1 do you consider to be complete sentences (see the file batch.csv) ?

Batch result
       french                               wolof
0          ki                                 cet
1          ki                           que voici
2       téeré                            amulette
3        tere                           interdire
4        dara                                rien
5         ree                                rire
6         daa                               encre
7       muumë                                muet
8         wér                              guérir
9         wër                       faire le tour
10       wéer                             adosser
11       géer                homme de noble caste
12      gisóo                       voir ensemble
13     baaxal                          rendre bon
14     tuurul  faire une offrande à une divinité.
15    Demagul               Il n'a pas encore été
16  Seeteegul          Il n'a 

In [89]:
new_sent_extraction.extract_sentences(group_range=(500, 1000), save=True, batch_size=100)


Which of the groups of batch number 1 do you consider to be complete sentences (see the file batch.csv) ?

Batch result
          french                                      wolof
0        baadolo                                     manant
1       mbaadolo                             état de manant
2           baax                                   être bon
3          mbaax                                      bonté
4            far                                      allié
5           mpar                                   alliance
6           nkor                                   trahison
7           boot                          porter sur le dos
8        bootaay            la manière de porter sur le dos
9           ñaaw                                       laid
10       ñaawaay                                 la laideur
11           saf                                    piquant
12        safaay                                  l'acidité
13        safaay                        

In [90]:
new_sent_extraction.extract_sentences(group_range=(600, 1000), save=True, batch_size=100)


Which of the groups of batch number 1 do you consider to be complete sentences (see the file batch.csv) ?

Batch result
      french                       wolof
0   xoolante                 se regarder
1       bëgg                       aimer
2   bëggante                     s'aimer
3   bëggante                       amour
4       nuyu                      saluer
5    nuyonte                   se saluer
6      duggu       s'imposer à quelqu'un
7   duggonte                     se lier
8   duggonte               s'acquoquiner
9   lekkantu    faire semblant de manger
10      gaaw                      rapide
11  gëmmentu               avoir sommeil
12        fo                       jouer
13        fo                         jeu
14        po                         jeu
15     fontu                       jouer
16     fontu          manquer de sérieux
17  fattarñi                   déboucher
18       fot           avaler de travers
19  fottarñi                    arracher
20    daggat       

ValueError: invalid literal for int() with base 10: ''

We provide an error let us add correction and repeat the last process.

In [154]:
new_sent_extraction.extract_sentences(save=True, batch_size=100)


Which of the groups of batch number 1 do you consider to be complete sentences (see the file batch.csv) ?
You cannot combine the same type of command: Type of commands are 'end mark' and 'upper'

Batch result
                                 french  \
0                   xale yile yarunañu.   
1                 jigéen jale laa wax !   
2                     xale buu laa wax.   
3                    jigéen jule demul.   
4                   xale yule yaruwuñu.   
5                    xam õga xale bee ?   
6                          jigéen jale.   
7                  téere yee laa jënd !   
8   waxtaanal ag nit kookii ci sa wet !   
9               gis õga jigéen ñooñii ?   
10                     mayal nag wile !   
11                 xam õga nit kookuu ?   
12               gis õga xale yooyule ?   
13            nit kookuu génn laa wax !   
14            gisóon õga jigéen joojule   
15              xamoon naa waa joojee !   
16      xammeewoon naa jigéen joojale !   
17               

In [164]:
new_sent_extraction.extract_sentences(save=True, batch_size=100)

Which of the groups of batch number 1 do you consider to be complete sentences (see the file batch.csv) ?

Batch result
                              french  \
0                Ya di ban jaõgkat ?   
1                Ya di ban jaõgkat ?   
2             Ya di seen ban xarit ?   
3                       Soo dee góor   
4                       Soo dee góor   
5                 Dil nitu réew mi !   
6                 Dil nitu réew mi !   
7                  Dil nitu nit ñi !   
8                  Dil nitu nit ñi !   
9               Yaa daan ganu Mustaf   
10                  Du sabar gu rëy.   
11           Xammee naa nagu Tugël !   
12           Xammee naa nagi Tugël !   
13            Gis naa doomu ndaw si.   
14          Gis naa doomi jigéen ja.   
15        Gis õga doomu jigéen jan ?   
16       Tann õga doomu benn jigéen.   
17       Tann õga doomu benn jigéen.   
18   Gis naa doomu nitu Gaanjóol gi.   
19  Gis naa këru xaritu Biraayim wi.   
20              Gis naa kër xaritam.   


In [8]:
new_sent_extraction.extract_sentences(save=True, batch_size=100)


Which of the groups of batch number 1 do you consider to be complete sentences (see the file batch.csv) ?

Batch result
                            french                                       wolof
0                 Wut leen ci biir                   Cherche-les à l'intérieur
1                          Démal !                                        Va !
2                         Génnal !                                      Sors !
3                          Génnéel                                 Fais sortir
4                       Kaayleen !                                     Venez !
5                  Demleen foofu !                              Allez là-bas !
6                        Nileen ka                                   Dites-lui
7                      Nileen leen                                  Dites-leur
8                      Gor gii dem                    Mais cet homme est parti
9                      Gor gii dem                         Cet homme qui a été
10         

In [9]:
new_sent_extraction.extract_sentences(save=True, batch_size=100)


Which of the groups of batch number 1 do you consider to be complete sentences (see the file batch.csv) ?

Batch result
                                french  \
0                      Yaa daawoon dem   
1                    Kooku daawoon dem   
2                 Góor gii daawoon dem   
3                           Su dee dem   
4                           Su dee dem   
5                             Su demee   
6                             Su demee   
7                             Su demee   
8                           Su dee dem   
9                           Su dee dem   
10                         Bi mu demee   
11                        Su dee Lawbe   
12                    Su dee Lawbe Rao   
13            Su kii dee doomu Birayim   
14                    Yaa doonkoon wax   
15                   Yaa doonkoon falu   
16                         Maa dul dem   
17                Wool góor gi dul dem   
18                    Wool kee dul dem   
19                    Wool kee dul dem  

We forgot to replace the "[ ?]" sign by " ?" and add it on the first sentences. But we did it after directly in the csv file. 

In [10]:
new_sent_extraction.extract_sentences(save=True, batch_size=100)

Which of the groups of batch number 1 do you consider to be complete sentences (see the file batch.csv) ?

Batch result
              french                      wolof
0        Keneen õga.            Tu es un autre.
1        Keneen õga.              Autre, tu es.
2        Keneen õga.           Un autre, tu es.
3       Ñeñeen lañu.        Il s'agit d'autres.
4       Ñeñeen lañu.          Il sont d'autres.
5       Ñeñeen lañu.          Autres, ils sont.
6           Menn la.             Il est unique.
7           Menn la.                Un, il est.
8       Ñooñu õgeen.         Vous êtes ceux-là.
9       Ñeneen lañu.        Il s'agit d'autres.
10      Ñeneen lañu.          Autres, ils sont.
11        Ñeneen la.          Ce sont d'autres.
12        Ñan lañu ?             Qui sont-ils ?
13          Ñan la ?              Qui sont-ce ?
14          Ñan la ?               Qui est-ce ?
15          Kan la ?               Qui est-ce ?
16        Mu di kii.       Mais c'est celui-ci.
17   Mu doon koo

We forgot to reply one mark in one of the first sentences. 

In [11]:
new_sent_extraction.extract_sentences(save=True, batch_size=100)


Which of the groups of batch number 1 do you consider to be complete sentences (see the file batch.csv) ?

Batch result
                            french  \
0                          ba itam   
1                               ni   
2                           ndegam   
3                          kon nag   
4                          kon nag   
5                               ci   
6                               ci   
7                               ak   
8                               ak   
9                               ak   
10                           balaa   
11                              te   
12                              te   
13                            itam   
14                              ag   
15                           walla   
16                           walla   
17                           waaye   
18                           wante   
19                           wante   
20                           wante   
21                         te itam   
22    

In [20]:
new_sent_extraction.extract_sentences(save=True, batch_size=100)


Which of the groups of batch number 1 do you consider to be complete sentences (see the file batch.csv) ?
You didn't provide a right command ! Please retry

Batch result
                                               french  \
0                                         Góor gi dem   
1                                              Mu dem   
2                                              Mu dem   
3                                         Ma ngii dem   
4                               Góor ñi deekoon nanu.   
5                                       Maa demulkoon   
6                                          Góor demul   
7                                          Góor demul   
8                                          Góor demul   
9                                       Góor gi demul   
10                           Xar yu ñuul dinañu feeñ.   
11                              Góor gi moo demulwoon   
12                                 Dem, rafetoon na !   
13                              

#### Drop duplicated translations

In [41]:
new_sent_extraction.load()

Let us show the number of duplicated translations.

In [42]:
pd.DataFrame(new_sent_extraction.extractions).duplicated().sum()

98

Let us drop the duplicated translations in our dataset.

In [43]:
new_sent_extraction.remove_duplicated_sentences(save=True)

Let us the final number of sentences that we got.

In [47]:
len(new_sent_extraction.extractions[new_sent_extraction.corpus_1])

2967

We finally extracted **2967** sentences.